In [6]:
import numpy as np
from PIL import Image
from pathlib import Path
from glob import glob
from tqdm import tqdm
from random import shuffle

In [7]:
base_path = Path('/home/stream/Downloads/DogsVsCats/PetImages')

In [3]:
!ls '/home/stream/Downloads/DogsVsCats/PetImages'

Cat  Dog


In [4]:
# X -> Pixels da imagem
# y -> Cat or Dog

In [11]:
classes = ['Cat', 'Dog']

In [6]:
def get_image(image_path, image_size=(224, 224)):
    X = np.array(Image.open(image_path).resize(image_size).convert('RGB')).flatten()/255.0
    y = classes.index(image_path.split("/")[-2])

#     print(X.shape)
    
    return X, y

In [8]:
images = glob('/home/stream/Downloads/DogsVsCats/PetImages/*/*.jpg')
shuffle(images)

In [9]:
from collections import Counter

In [18]:
X, y = [], []
for image in tqdm(images[:2000]):
    xx, yy = get_image(image, image_size=(128, 128))
    X.append(xx)
    y.append(yy)

100%|██████████████████████████████████████████████| 2000/2000 [00:03<00:00, 575.39it/s]


In [19]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

In [20]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

In [21]:
pipeline = Pipeline([('scaler', StandardScaler()), ('svc', SVC())])

In [22]:
pipeline.fit(X_train, y_train)

Pipeline(steps=[('scaler', StandardScaler()), ('svc', SVC())])

In [23]:
pipeline.score(X_test, y_test)

0.618

In [24]:
# --

In [1]:
import torch
from torch.utils.data import Dataset, DataLoader

import albumentations as A
from albumentations.pytorch import ToTensorV2

/home/stream/.pyenv/versions/3.9.16/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
train_transform = A.Compose(
    [
        A.Resize(224, 224),
        A.Normalize(),
#         A.HorizontalFlip(p=0.5),
#         A.RandomBrightnessContrast(p=0.3),
        ToTensorV2(),
    ]
)

val_transform = A.Compose(
    [
        A.Resize(224, 224),
        A.Normalize(),
        ToTensorV2(),
    ]
)

In [3]:
class CatVsDogsDataset(Dataset):
    def __init__(self, images, classes=[], transform=None):
        self.images = images
        self.classes = classes
        self.transform = transform
    
    def __getitem__(self, index):
        image_path = self.images[index]
        image = np.array(Image.open(image_path).convert('RGB'))
        
        if self.transform:
            X = self.transform(image=image)['image']
            
        y = self.classes.index(image_path.split("/")[-2])
#         y = torch.Tensor(y])
        
        return X, y, image_path
    
    def __len__(self):
        return len(self.images)

In [8]:
images = glob('/home/stream/Downloads/DogsVsCats/PetImages/*/*.jpg')
shuffle(images)

In [9]:
train_images = images[:20000]
val_images = images[20000:]

In [12]:
train_dataset = CatVsDogsDataset(train_images, classes=classes, transform=train_transform)
val_dataset = CatVsDogsDataset(val_images, classes=classes, transform=val_transform)

In [13]:
train_dataloader = DataLoader(dataset=train_dataset, batch_size=16, shuffle=True)
val_dataloader = DataLoader(dataset=val_dataset, batch_size=1)

In [40]:
from torchvision.models import mobilenet_v2
import torch.nn as nn

In [58]:
del model

In [59]:
model = mobilenet_v2(weights='IMAGENET1K_V1')

In [60]:
model.classifier[-1] = nn.Linear(in_features=1280, out_features=2, bias=True)

In [61]:
model = model.cuda()

In [62]:
criterion = nn.CrossEntropyLoss()

In [65]:
epochs = 3

In [66]:
device = 'cuda'

In [67]:
from torch.optim import AdamW
import torchmetrics

In [68]:
optimizer = AdamW(model.parameters(), lr=1e-4)

In [69]:
metric = torchmetrics.Accuracy(task='multiclass', num_classes=2).to(device)

for epoch in range(epochs):
    model.train()
    for image, label, _ in tqdm(train_dataloader, total=len(train_dataloader)):
        image = image.to(device)
        label = label.to(device)
        
        optimizer.zero_grad()
        
        pred = model(image)
        loss = criterion(pred, label)
        loss.backward()
        optimizer.step()
        
        metric(pred, label)
    acc = metric.compute()
    
    print(f'train_loss: {loss}, train_acc: {acc}')
    metric.reset()
    
    model.eval()
    for image, label, _ in tqdm(val_dataloader, total=len(val_dataloader)):
        image = image.to(device)
        label = label.to(device)
        
        pred = model(image)
        
        loss = criterion(pred, label)
        
        metric(pred, label)
        
    acc = metric.compute()
    print(f'val_loss: {loss}, val_acc: {acc}')
    metric.reset()

100%|███████████████████████████████████████████████| 1250/1250 [01:22<00:00, 15.14it/s]


train_loss: 0.07808191329240799, train_acc: 0.970550000667572


100%|██████████████████████████████████████████████| 4998/4998 [00:37<00:00, 132.04it/s]


val_loss: 1.0847986231965479e-05, val_acc: 0.982192873954773


100%|███████████████████████████████████████████████| 1250/1250 [01:22<00:00, 15.22it/s]


...

val_loss: 1.7881377516459906e-06, val_acc: 0.9851940870285034


100%|███████████████████████████████████████████████| 1250/1250 [01:22<00:00, 15.23it/s]


train_loss: 0.07246063649654388, train_acc: 0.9914500117301941


100%|██████████████████████████████████████████████| 4998/4998 [00:37<00:00, 131.93it/s]

val_loss: 0.0, val_acc: 0.9835934638977051


In [72]:
model

MobileNetV2(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
        )
        (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(96, eps=

In [75]:
x = torch.randn(1, 3, 224, 224, requires_grad=True).cuda()
torch_out = model(x)
torch.onnx.export(
    model,
    x,
    "catsvsdogs.onnx",
    export_params=True,
    opset_version=13,
    do_constant_folding=True,
    input_names=["input"],
    output_names=["output"],
    dynamic_axes={"input": {0: "batch_size"}, "output": {0: "batch_size"}},
)

In [76]:
!ls

catsvsdogs.onnx  explore.ipynb	main.py
